In [1]:
import nltk
import pandas as pd
import numpy as np
from processing import *
from sentiment import *

In [2]:
# TODO: pindah ke sqlite
dataset = pd.read_csv("dataset/training_dataset.csv")
all_pos = dataset[dataset['label'] == 1]
all_neg = dataset[dataset['label'] == 0]


# * Training data: 80% pertama
# * Testing data: 20% terakhir
train_pos = all_pos[int(len(all_pos)*0.8):]
train_neg = all_neg[int(len(all_neg)*0.8):]
test_pos = all_pos[:int(len(all_pos)*0.2)]
test_neg = all_neg[:int(len(all_neg)*0.2)]

train_x = train_pos.append(train_neg, ignore_index=True)
test_x = test_pos.append(test_neg, ignore_index=True)

train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

In [4]:
x = train_x['text']
y = train_y
kamus_frek = kamus_freq(x, y)

In [ ]:
logprior, loglikelihood = train_nbc(kamus_frek, train_x, train_y)

In [ ]:
hasil_uji = test_nbc(x, y, logprior, loglikelihood)
hasil_uji

In [ ]:
berita = open('berita.txt', 'r')
berita = berita.read()
berita = normalisasi(berita)

In [ ]:
predict_nbc(berita, logprior, loglikelihood)

In [ ]:
predict_perkata(berita, logprior, loglikelihood)